In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
files.upload();

Saving Lecture8-Data.xlsx to Lecture8-Data.xlsx


In [ ]:
df = pd.read_excel("Lecture8-Data.xlsx", sheet_name="Historical_Data")
train_data =df.drop(columns=['Item#','Demand','Price'])
train_data.head()

,Beginning_of_Season,Weekend,Event_Length,Morning,Branded,Color_Popularity,ConcurrentEvents,Number_Competing_Styles_in_Event,Relative_Price_of_Competing_Styles,Num_Branded_Events12,Brand_MSRP_Index,Bottoms,Tops,Dresses
0,0,0,1,1,1,0.018859,3,29,0.953903,0,1.311367,0,0,1
1,0,0,2,1,1,0.114444,3,9,1.000000,2,0.636349,0,0,1
2,1,0,2,1,1,0.114444,4,38,0.884913,4,0.820957,0,1,0
3,0,0,3,1,1,0.310412,3,13,1.013009,2,0.616465,0,0,0
4,1,0,2,1,0,0.310412,4,25,0.835286,0,0.701432,0,0,0


In [ ]:
y_train = df['Demand']  # Target variable
y_train.head()

0    2854.00000
1    2817.00000
2    5089.00000
3    6906.00000
4    3376.43338
Name: Demand, dtype: float64

In [ ]:
files.upload();

Saving A5.xlsx to A5.xlsx


In [ ]:
test_data= pd.read_excel("A5.xlsx")
test_data.head()

,Beginning_of_Season,Weekend,Event_Length,Morning,Branded,Color_Popularity,ConcurrentEvents,Number_Competing_Styles_in_Event,Relative_Price_of_Competing_Styles,Num_Branded_Events12,Brand_MSRP_Index,Bottoms,Tops,Dresses
0,1,0,2,0,1,0.17,2,3,0.833333,2,0.87,0,1,0
1,1,0,2,0,1,0.17,2,3,1.000000,2,0.87,0,1,0
2,1,0,2,0,1,0.17,2,3,1.166667,2,0.87,0,1,0
3,1,0,2,0,0,0.65,2,3,0.833333,1,0.33,0,1,0
4,1,0,2,0,0,0.65,2,3,1.000000,1,0.33,0,1,0


In [ ]:
regr = linear_model.LinearRegression()
regr.fit(train_data, y_train)

predicted_demand = regr.predict(test_data)

predicted_demand_df = pd.DataFrame({"Predicted_Demand": predicted_demand})
predicted_demand_df.head(10)

,Predicted_Demand
0,5993.945379
1,5793.583183
2,5593.220986
3,5725.942559
4,5525.580362
5,5325.218166
6,5630.114342
7,5429.752146
8,5229.389949


In [ ]:
!pip install -q pyomo
!apt-get install -y -qq glpk-utils
!which glpsol

from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition

/usr/bin/glpsol


$$
\begin{aligned}
\operatorname{Max} & \sum_{i \in N} \sum_{j \in M} p_j \widehat{D}_{i j} x_{i j} \\
\text { s.t } \\
& \sum_{j \in M} x_{i j}=1 \quad \forall i \in N \\
& \sum_{i \in N} \sum_{j \in M} p_j x_{i j}=k \\
& x_{i j} \in\{0,1\}\\
\end{aligned}
$$


In [ ]:
# List of k values- Total budget constraint
k_values = [75, 80, 85, 90, 95, 100, 105]

# Initialize variables to store maximum revenue and corresponding k value
max_revenue = float('-inf')
optimal_k = None

# Iterate over each k value
for k_value in k_values:

  #creating the model object
  model = ConcreteModel()

  # List of items
  items = ['A', 'B', 'C']

  # List of possible prices
  prices = [25, 30, 35]

  # Predicted demand values
  predicted_demand = [5993.945379, 5793.583183, 5593.220986, 5725.942559, 5525.580362, 5325.218166, 5630.114342, 5429.752146, 5229.389949]

  # Decision variables
  model.x = Var(items, prices, within=Binary) #Items and prices are binary, meaning they can only take on values of 0 or 1

  # Objective function: maximize total revenue
  model.obj = Objective(expr=sum(prices[j] * predicted_demand[i*3+j] * model.x[items[i], prices[j]] for i in range(len(items)) for j in range(len(prices))),
                      sense=maximize)

  # Define ConstraintList for exactly one price per item
  model.exactly_one_price_per_item = ConstraintList()

  # Cmodel.exactly_one_price_per_item = ConstraintList()
  for item in items:
     model.exactly_one_price_per_item.add(sum(model.x[item, p] for p in prices) == 1)


  # Constraint: Total price of assigned items must be equal to the budget
  model.total_budget_constraint = Constraint(expr=sum(prices[j] * sum(model.x[items[i], prices[j]] for i in range(len(items))) for j in range(len(prices))) == k_value)

  # Solve the optimization problem
  solver = SolverFactory('glpk', executable='/usr/bin/glpsol')
  solver.solve(model)

  total_revenue = sum(prices[j] * predicted_demand[i*3+j] * value(model.x[items[i], prices[j]]) for i in range(len(items)) for j in range(len(prices)))

 # Check if revenue is maximum
  if total_revenue > max_revenue:
      max_revenue = total_revenue
      optimal_k = k_value

  # Output the maximum revenue and corresponding k value
  print(f"Maximum revenue: {max_revenue}")
  print(f"Optimal k value: {optimal_k}")

  # Output the results
  for i in range(len(items)):
      for j in range(len(prices)):
          if value(model.x[items[i], prices[j]]) > 0.5:
              print(f"Item {items[i]} is assigned a price of ${prices[j]}")

  print("Total revenue (objective value):", total_revenue)


print('\n')
print(f"Maximum revenue: {max_revenue}")
print(f"Optimal k value: {optimal_k}")

Maximum revenue: 433750.05700000003
Optimal k value: 75
Item A is assigned a price of $25
Item B is assigned a price of $25
Item C is assigned a price of $25
Total revenue (objective value): 433750.05700000003
Maximum revenue: 457708.918015
Optimal k value: 80
Item A is assigned a price of $30
Item B is assigned a price of $25
Item C is assigned a price of $25
Total revenue (objective value): 457708.918015
Maximum revenue: 480327.7649
Optimal k value: 85
Item A is assigned a price of $30
Item B is assigned a price of $30
Item C is assigned a price of $25
Total revenue (objective value): 480327.7649
Maximum revenue: 502467.47073
Optimal k value: 90
Item A is assigned a price of $30
Item B is assigned a price of $30
Item C is assigned a price of $30
Total revenue (objective value): 502467.47073
Maximum revenue: 524422.70975
Optimal k value: 95
Item A is assigned a price of $35
Item B is assigned a price of $30
Item C is assigned a price of $30
Total revenue (objective value): 524422.7097

**Compare the results of the Regression Tree and Linear Regression methods. Did the optimal prices change?**

From lecture 8, it was found that the optimal prices when using **Regression Tree** for items A, B and C were 35, 30 and 25 respectively with a total revenue of 874347 when K=90. In this assignment, when K=90 when using **Linear Regression**, the prices for items A, B and C were found to be 30 each with a total revenue of 524422.

It does seem like there might be a discrepancy between the optimal prices and total revenue obtained from the decision tree model and the linear regression model when K=90. This might be due to the different ways of training a model.

Question 2


In [ ]:
# List of k values- Total budget constraint
k_values = [75, 80, 85, 90, 95, 100, 105]

# Initialize variables to store maximum revenue and corresponding k value
max_revenue = float('-inf')
optimal_k = None

# Iterate over each k value
for k_value in k_values:
  print(f"K value: {k_value}")

  model = ConcreteModel()

  # Set of items
  items = ['A', 'B', 'C']

  # Set of possible prices
  prices = [25, 30, 35]

  # Predicted demand values (replace with your actual predicted demand values)
  predicted_demand = [5993.945379, 5793.583183, 5593.220986, 5725.942559, 5525.580362, 5325.218166, 5630.114342, 5429.752146, 5229.389949]

  # Decision variables
  model.x = Var(items, prices, within=Binary)

  # Objective function: maximize total revenue
  model.obj = Objective(expr=sum(prices[j] * predicted_demand[i*3+j] * model.x[items[i], prices[j]] for i in range(len(items)) for j in range(len(prices))),
                      sense=maximize)

  # Define ConstraintList for exactly one price per item
  model.exactly_one_price_per_item = ConstraintList()
  for item in items:  # Iterate over the items directly
      if item == 'A' or item == 'B':
          # If item is A or B, ensure it is not assigned price 35
          model.exactly_one_price_per_item.add(model.x[item, 35]== 0)
          model.exactly_one_price_per_item.add(sum(model.x[item, p] for p in prices if p != 35) == 1)
      elif item == 'C':
          # If item is C, ensure it is not assigned price 25
          model.exactly_one_price_per_item.add(model.x[item, 25]== 0)
          model.exactly_one_price_per_item.add(sum(model.x[item, p] for p in prices if p != 25) == 1)
      else:
          # For other items, enforce normal pricing constraint
          model.exactly_one_price_per_item.add(sum(model.x[item, p] for p in prices) == 1)

  # Constraint: Total price of assigned items must be equal to the budget
  model.total_budget_constraint = Constraint(expr=sum(prices[j] * sum(model.x[items[i], prices[j]] for i in range(len(items))) for j in range(len(prices))) == k_value)

  # Solve the optimization problem
  solver = SolverFactory('glpk', executable='/usr/bin/glpsol')
  results = solver.solve(model)

  # Check if solver successfully solved the problem
  if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):

    # Compute total revenue
    total_revenue = sum(prices[j] * predicted_demand[i*3+j] * value(model.x[items[i], prices[j]]) for i in range(len(items)) for j in range(len(prices)))

    # Check if revenue is maximum
    if total_revenue > max_revenue:
      max_revenue = total_revenue
      optimal_k = k_value

    # Output the results
    for i in range(len(items)):
      for j in range(len(prices)):
        if value(model.x[items[i], prices[j]]) > 0.5:
          print(f"Item {items[i]} is assigned a price of ${prices[j]}")

    print("Total revenue (objective value):", total_revenue)
  else:
    print("Solver failed to find an optimal solution")


print('\n')
print(f"Maximum revenue: {max_revenue}")
print(f"Optimal k value: {optimal_k}")

K value: 75
Solver failed to find an optimal solution
K value: 80
Item A is assigned a price of $25
Item B is assigned a price of $25
Item C is assigned a price of $30
Total revenue (objective value): 455889.76283
K value: 85
Item A is assigned a price of $30
Item B is assigned a price of $25
Item C is assigned a price of $30
Total revenue (objective value): 479848.623845
K value: 90
Item A is assigned a price of $30
Item B is assigned a price of $30
Item C is assigned a price of $30
Total revenue (objective value): 502467.47073
K value: 95
Item A is assigned a price of $30
Item B is assigned a price of $30
Item C is assigned a price of $35
Total revenue (objective value): 522603.554565
K value: 100
Solver failed to find an optimal solution
K value: 105
Solver failed to find an optimal solution


Maximum revenue: 522603.554565
Optimal k value: 95
